In [1]:
import requests
import datetime
from urllib.parse import urlencode
import pandas as pd
import time
import base64

# Acceso a la API de Spotify

In [2]:
file = open(f'../sptoken.txt')
client_secret = file.readlines()[0].strip('\n')

headers = {
    'Authorization': f'Bearer {client_secret}'
}

In [3]:
client_id = 'd48abffc774f4dd7af256702396c6ac3'

In [4]:
client_creds = f'{client_id}:{client_secret}'

client_cred_b64 = base64.b64encode(client_creds.encode())

In [5]:
# Se hace la request para que nos den el token y poder acceder a los datos

token_url = 'https://accounts.spotify.com/api/token'
method = 'POST'

token_data = {
        'grant_type': 'client_credentials' 
}

token_header = {
        'Authorization': f'Basic {client_cred_b64.decode()}'
}

In [6]:
r = requests.post(token_url, data=token_data, headers=token_header)
token_response_data = r.json()
access_token = token_response_data['access_token']
access_token

'BQCOL7_Q0HQakzN91UmQdnGrPkCeGQVX3K8PmoqCYXzs661Yqp0UTD3gUfD1daeTGwxLkMSmkB0qwb4L0Oo'

In [7]:
r

<Response [200]>

In [8]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='playlists', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_playlist_tracks(self, query, version='v1', resource_type='playlists', item='tracks'):
        headers = self.get_resource_header()
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{query}/{item}'
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def search_by_name(self, query, search_type='playlist'): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

# Obtención de las playlist

In [9]:
headers = {
    "Authorization": f"Bearer {access_token}"
}

endpoint = "https://api.spotify.com/v1/playlist"
data = urlencode({"q": "Time", "type": "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

q=Time&type=track
404


In [10]:
# inicializamos el objeto de spotify

client_id = 'd48abffc774f4dd7af256702396c6ac3'
spotify = SpotifyAPI(client_id, client_secret)

In [11]:
# Queremos buscar las canciones más destacadas de la decada de los 60's

sixtys = spotify.search_by_name("All Out 60s", search_type="playlist")
lst_plst = sixtys['playlists']['items']
lst_plst[:2]

[{'collaborative': False,
  'description': 'The biggest songs of the 1960s.\n',
  'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DXaKIA8E7WcJj'},
  'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXaKIA8E7WcJj',
  'id': '37i9dQZF1DXaKIA8E7WcJj',
  'images': [{'height': None,
    'url': 'https://i.scdn.co/image/ab67706f00000003879d521d998fc7dda0cf1b1d',
    'width': None}],
  'name': 'All Out 60s',
  'owner': {'display_name': 'Spotify',
   'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
   'href': 'https://api.spotify.com/v1/users/spotify',
   'id': 'spotify',
   'type': 'user',
   'uri': 'spotify:user:spotify'},
  'primary_color': None,
  'public': None,
  'snapshot_id': 'MTY1MDEwNjg0NSwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl',
  'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXaKIA8E7WcJj/tracks',
   'total': 150},
  'type': 'playlist',
  'uri': 'spotify:playlist:37i9dQZF1DXaKIA8E7WcJj'},
 {'colla

In [12]:
df = pd.DataFrame(lst_plst)
df.head()

,collaborative,description,external_urls,href,id,images,name,owner,primary_color,public,snapshot_id,tracks,type,uri
0,False,The biggest songs of the 1960s.\n,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DXaKIA8E7WcJj,"[{'height': None, 'url': 'https://i.scdn.co/im...",All Out 60s,"{'display_name': 'Spotify', 'external_urls': {...",None,None,MTY1MDEwNjg0NSwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMD...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:37i9dQZF1DXaKIA8E7WcJj
1,False,The essentials from the decade,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DX9kVlnA5Si6s,"[{'height': None, 'url': 'https://i.scdn.co/im...",All Out 70s,"{'display_name': 'Spotify', 'external_urls': {...",None,None,MTY1MDEwNjgyMywwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMD...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:37i9dQZF1DX9kVlnA5Si6s
2,False,"The best songs from the 1960s, 70s and 80s in ...",{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/44aLSiKMT...,44aLSiKMThIYqDg6Eh5OSl,"[{'height': None, 'url': 'https://i.scdn.co/im...","All Out 60's, 70's and 80's","{'display_name': 'Cameron Greenwood', 'externa...",None,None,MjI5LDMzMDY0ZjkzNjExZDUxMjg3YWZmMjU0OGY4MjI0Mj...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:44aLSiKMThIYqDg6Eh5OSl
3,False,,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/0yaWIghxW...,0yaWIghxWNuxpnF3OMHa0R,"[{'height': 640, 'url': 'https://mosaic.scdn.c...","All out 40s, 50s & 60s","{'display_name': 'bo_nanza', 'external_urls': ...",None,None,MzI5LGRjNzQwMzZjNDg5YWU3YWUwMTI0ZGQ2N2NhYTFiM2...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:0yaWIghxWNuxpnF3OMHa0R
4,False,"The golden era of rock n roll &amp; soul, incl...",{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/2FVpT1U5v...,2FVpT1U5v0krrwlr4qOOYu,"[{'height': None, 'url': 'https://i.scdn.co/im...",All Out 60s,"{'display_name': 'CoLiNdAbEaSt', 'external_url...",None,None,MTY2LDdmY2NjMmZlNjI3ZWE2ZTVhODY1MzhlMjMwYjdkMG...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:2FVpT1U5v0krrwlr4qOOYu


En la búsqueda, hay que cerciorarse de que el autor es el propio spotify (con el fin de no dar cabida a las opiniones
de personas aisladas)

In [13]:
def get_creator(dic):
    return dic['display_name']

In [14]:
def get_spoti_playlist(df):
    df.owner = list(map(get_creator, list(df.owner)))
    df = df[df['owner'] == 'Spotify']
    sxt_plst = df.loc[:0]
    return sxt_plst

In [15]:
sxt_plst = get_spoti_playlist(df)

In [16]:
#hay que hacer los endpoint

pl_id = str(sxt_plst['id']).split('    ')[-1].split('\n')[0]

def get_df(pl_id):
    data = []

    sxt_tracks = spotify.get_playlist_tracks(pl_id)

    for i in range(len(sxt_tracks['items'])):
        art_lst = []
        sng_lst = []
        alb_lst = []
        dt_lst = []

        artista = sxt_tracks['items'][i]['track']['artists'][0]['name']
        art_lst.append(artista)

        cancion = sxt_tracks['items'][i]['track']['name']
        sng_lst.append(cancion)

        album = sxt_tracks['items'][i]['track']['album']['name']
        alb_lst.append(album)

        lanzamiento = sxt_tracks['items'][i]['track']['album']['release_date']
        dt_lst.append(lanzamiento)

        fila = []
        fila += art_lst + sng_lst + alb_lst + dt_lst

        data.append(fila)

    cols = ['Artista', 'Titulo', 'Album', 'Fecha']

    df = pd.DataFrame(data, columns=cols)
    return df

In [17]:
df_sxt = get_df(pl_id)
df_sxt

,Artista,Titulo,Album,Fecha
0,The Temptations,My Girl,The Temptations Sing Smokey,1965-01-01
1,Percy Sledge,When a Man Loves a Woman,When a Man Loves a Woman,2018-05-05
2,The Rolling Stones,(I Can't Get No) Satisfaction - Mono Version,Hot Rocks (1964-1971),1971-12-20
3,Sam Cooke,(What A) Wonderful World,The Man Who Invented Soul,2000-09-26
4,Van Morrison,Brown Eyed Girl,Blowin' Your Mind!,1967-09
...,...,...,...,...
95,Ike & Tina Turner,River Deep - Mountain High,Tina!,2008-09-30
96,Herb Alpert & The Tijuana Brass,This Guy's In Love With You,The Beat Of The Brass,1968-05-01
97,Four Tops,Baby I Need Your Loving,Four Tops,1965-01-01
98,Bee Gees,Massachusetts (2008 Remastered LP Version),"Best Of Bee Gees, Vol. 1",1969-06-01
